In [13]:
import torch
from torch.nn.utils import prune

from transformers import AutoTokenizer, OPTForCausalLM, pipeline
from datasets import load_dataset

from calculate_mask import calculate_mask
from inverse_hessian import inverse_hessian
from input_prehooks import put_input_hooks

In [14]:
#DEVICE
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#Load dataset
dataset = load_dataset('c4', 'en', streaming=True)
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")
# Load model with pre-trained head
model = OPTForCausalLM.from_pretrained("facebook/opt-125m", output_attentions=True, output_hidden_states=True)
# Load genrator
generator = pipeline('text-generation', model="facebook/opt-125m")
# Create calibration data
calibration_data = []
for i, data in enumerate(iter(dataset['train'])):
    if i > 127:
        break
    tokenized = tokenizer.encode(data['text'], return_tensors="pt", padding="max_length", truncation=True, max_length=1024)
    calibration_data.append(tokenized)
calibration_data = torch.squeeze(torch.stack(calibration_data)).to(device=device)

In [15]:
# First, put in forward hooks
features = {}
put_input_hooks(model=model, features=features)

# Run calibration data through model at first to calculate features dictionary with
# input tensors to each intermediate layer
model(calibration_data)

# function to get module name from parameter name
def get_module_name(param_name):
    if param_name[-5:] == ".bias":
        return param_name[:-5], "bias"
    elif param_name[-7:] == ".weight":
        return param_name[:-7], "weight"
    else:
        return None, None

In [ ]:
# Re-load model with pre-trained head
model = OPTForCausalLM.from_pretrained("facebook/opt-125m", output_attentions=True, output_hidden_states=True)

# make a dictionary to access module by name
module_lookup_dict = {}
for module_name, module_iter in model.named_modules():
    module_lookup_dict[module_name] = module_iter
EPSILON = 0.0001
SPARSENESS = .9
B = 128
Bs = 64

In [ ]:
from tqdm import tqdm
layer_blacklist = ['model.decoder.embed_tokens.weight', 'model.decoder.embed_tokens.bias',
'model.decoder.embed_positions.weight']

# Using calibration data (inputs to each intermediate weight layer)
# Iterate through named parameters, calculate inverse hessian and calculate mask

# without this
param_lookup_dict = {}
param_names = []
for name, param in model.named_parameters():
    param_names.append(name)
    param_lookup_dict[name] = param

with torch.no_grad():
    for name in tqdm(param_names):
        param = param_lookup_dict[name]

        # skip the embed layer
        if name in layer_blacklist:
            continue
        
        # skip norms which have 1 dimension
        if len(param.shape) < 2:
            continue

        module_name, param_type = get_module_name(name)

        # apply to weight and bias layers
        if param_type == "weight" or param_type == "bias":
            # input to parameter
            layer_input = features[module_name][0]
            #print(name)

            # calculate inverse hessian
            # check if input is flattened e.g. from 8,512,768 to 4096,768
            if len(layer_input.shape) == 2:
                inv_hess = inverse_hessian(torch.transpose(layer_input, 0, 1), epsilon=EPSILON, 
                flattened=True)

            else:
                inv_hess = inverse_hessian(torch.transpose(layer_input, 1, 2), epsilon=EPSILON,
                flattened=False)

            # inv_hess = inverse_hessian(layer_input, epsilon=EPSILON)
            # print(f"hessian shape: {inv_hess.shape}")

            # calculate mask
            mask = calculate_mask(W=param, H_inv=inv_hess, p=SPARSENESS, B=B, Bs=Bs)
            
            # get module from lookup dictionary by module name
            module = module_lookup_dict[module_name]
            # apply mask
            prune.custom_from_mask(module=module, name=param_type, mask=mask)

  0%|          | 0/196 [00:00<?, ?it/s]c:\Users\Aaquib\OneDrive\Documents\ICLR\inverse_hessian.py:30: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\BatchLinearAlgebra.cpp:2121.)
  triangle_inv = torch.triangular_solve(triangle, identity, upper=True)[0]
100%|██████████| 196/196 [02:09<00:00,  1.52it/s]


In [ ]:
torch.cuda.is_available()

False

In [ ]:
for name, param in model.named_parameters():
    print(name)

model.decoder.embed_tokens.weight
model.decoder.embed_positions.weight
model.decoder.final_layer_norm.weight
model.decoder.final_layer_norm.bias
model.decoder.layers.0.self_attn.k_proj.bias
model.decoder.layers.0.self_attn.k_proj.weight_orig
model.decoder.layers.0.self_attn.v_proj.bias
model.decoder.layers.0.self_attn.v_proj.weight_orig
model.decoder.layers.0.self_attn.q_proj.bias
model.decoder.layers.0.self_attn.q_proj.weight_orig
model.decoder.layers.0.self_attn.out_proj.bias
model.decoder.layers.0.self_attn.out_proj.weight_orig
model.decoder.layers.0.self_attn_layer_norm.weight
model.decoder.layers.0.self_attn_layer_norm.bias
model.decoder.layers.0.fc1.bias
model.decoder.layers.0.fc1.weight_orig
model.decoder.layers.0.fc2.bias
model.decoder.layers.0.fc2.weight_orig
model.decoder.layers.0.final_layer_norm.weight
model.decoder.layers.0.final_layer_norm.bias
model.decoder.layers.1.self_attn.k_proj.bias
model.decoder.layers.1.self_attn.k_proj.weight_orig
model.decoder.layers.1.self_attn

In [ ]:
input1 = tokenizer("Hello, my dog is cute", return_tensors="pt", padding="max_length", truncation=True)
input2 = tokenizer("What the fuck did you just fucking say about me, you little bitch?", return_tensors="pt", padding="max_length", truncation=True)
output = model.generate(input1.input_ids, max_length=100, num_return_sequences=1, temperature=0.9, top_p=0.95)
tokenizer.decode(output[0], skip_special_tokens=True)

'Hello, my dog is cute, my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog is my dog'